# Personal Agent using Memory and websearch

A specialized Strands agent that personalizes the answers based on websearch and memory.

In this notebook, we will:
- Explore the capabilities of a memory-powered Strands agent.
- Learn how to store, retrieve, and list memories.
- Understand how to perform web searches via the agent.
- Interact with the agent in an interactive loop.


### Usage Examples

Storing memories:
```
Remember that I prefer tea over coffee
```

Retrieving memories:
```
What do I prefer to drink?
```

Listing all memories:
```
Show me everything you remember about me
```

### Tips for Memory Usage

- Be explicit when asking the agent to remember information
- Use specific queries to retrieve relevant memories
- Memory persistence enables more natural and contextual conversations

## Introduction and Setup

Let's begin by setting up the environment and importing the required libraries.

In [ ]:
# Install the required packages
!pip install -r requirements.txt

In [1]:
# Import Required Libraries
import os
from strands import Agent, tool
from strands_tools import mem0_memory
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException, RatelimitException

## Environment Variable Configuration

To enable the agent's functionality, we need to configure environment variables for AWS credentials and OpenSearch. These variables are used for memory storage and retrieval.

In [ ]:
# Set up environment variables for AWS credentials and OpenSearch
# REPLACE WITH YOUR CREDENTIALS
os.environ["OPENSEARCH_HOST"] = "your-opensearch.us-west-2.aoss.amazonaws.com"

os.environ["AWS_REGION"] = "your-region" 
os.environ['AWS_ACCESS_KEY_ID'] = "your-access-key-id" 
os.environ['AWS_SECRET_ACCESS_KEY'] = "your-secret-access-key"

## Define System Prompt

The `SYSTEM_PROMPT` variable defines the behavior and capabilities of the memory agent. This prompt guides the agent to provide personalized responses based on stored memories and perform web searches when necessary.

In [3]:
# Define a focused system prompt for memory operations
SYSTEM_PROMPT = """You are a helpful personal assistant for a user. Your task is to assist the user by providing personalized responses based on their history. 

Capabilities:
- You can store information using the mem0_memory tool (action="store").
- You can retrieve relevant memories using the mem0_memory tool (action="retrieve").
- You can use duckduckgo_search to find information on the web.

Key Rules:
- Be conversational and natural in your responses.
- Always retrieve memories before responding to the user and use them to inform your response.
- Store any new user information and user preferences in mem0_memory.
- Only share relevant information.
- Politely indicate when you don't have the information.
"""

## Define Web Search Tool

The `websearch` tool using [Duckduckgo Search API](https://github.com/deedy5/duckduckgo_search) function allows the agent to perform web searches. This function handles exceptions and returns search results or appropriate error messages.

In [4]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

## Create Memory Agent

We will now initialize the memory-focused agent using the defined tools and system prompt. The Strands agent is capable of:
1. Storing and retrieving memories based on context. It uses memory to create more personalized and contextual AI interactions.
2. Performing web searches using DuckDuckGo to give updated information.

In [5]:
# Create an agent with memory, websearch tool
USER_ID = "new_user" # Replace with actual user ID

memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[mem0_memory, websearch],
)

## Demonstrate Memory Operations

The following examples demonstrate how to store, retrieve, and list memories using the memory agent.

- **store**: Save important information for later retrieval
  - Store user preferences
  - Remember important facts
  - Maintain conversation context

- **retrieve**: Access relevant memories based on queries
  - Find previously stored information
  - Provide personalized responses based on user history

- **list**: View all stored memories
  - See what information has been retained
  - Audit stored memories

In [ ]:
# Store initial memories to demonstrate retrieval
memory_agent.tool.mem0_memory(
    action="store", content=f"The user's name is {USER_ID}.", user_id=USER_ID
)
memory_agent.tool.mem0_memory(
    action="store", 
    content="I like to drink tea more than coffee.", 
    user_id=USER_ID
)

In [ ]:
# Retrieve memories
retrieved_memories = memory_agent.tool.mem0_memory(
    action="retrieve", query="What is the user's name?", user_id=USER_ID
)
print("Retrieved Memories:", retrieved_memories)

In [ ]:
# Retrieve memories about preferences
memory_agent.tool.mem0_memory(
    action="retrieve",
    query="What are the my drink preferences?",
    user_id=USER_ID
)

In [ ]:
# Ask the agent a question
response = memory_agent("What are the events happening in the New York today?")
print(response)

In [ ]:

# List all stored memories
print("All Stored Memories:")
all_memories = memory_agent.tool.mem0_memory(
    action="list", user_id=USER_ID
)

## Interactive Agent Usage

Finally, we provide an interactive loop for users to interact with the memory agent. Users can store new memories, retrieve existing ones, or list all stored memories.

To test interactive usage: 
1. Install the requirements: `pip install -r requirements.txt`
1. Run the python file `personal_agent_with_memory.py`. 

## Conclusion

This notebook demonstrates how to create a personal agent with memory capabilities using the Strands framework. The agent can:

1. Store information about the user
2. Retrieve relevant memories based on context
3. Search the web for additional information
4. Provide personalized responses

By combining these capabilities, the agent can maintain context across conversations and provide more personalized assistance over time.